In [1]:
%matplotlib widget
import os
import copy
import warnings
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import ipywidgets as widgets

from scipy.io import loadmat
from ipywidgets import interact
from ipywidgets import FileUpload
from IPython.display import display, HTML, clear_output, Javascript

global ax
global fig
global idx_list
global xs_values 
global ys_values
global plot_output
global saved_df_list
global new_markers_loc

DATASET_FOLDER = './dataset'
OUTPUT_FOLDER = './annotated_data'
SAMPLE_RATE = 50 # in Hz

# This needs to change if sample rate is changed
# L means milliseconds -> formula 1/SAMPLE_RATE = "xL"
# 1/50Hz = 0.02 = "20L"

SAMPLE_RATE_STRING = '20L' # equivalent to 50Hz. 
PEAK_DET_THRESHOLD = .02
DIFF_TRSHD = .001

warnings.filterwarnings('ignore')
display(HTML("<style>.container { width:100% !important; }</style>"))

OFFSET = 0.02

def peakdet(df, interval_offset, delta, x=None):    
    """
    Converted from MATLAB script at http://billauer.co.il/peakdet.html
    
    Returns two arrays
    
    function [maxtab, mintab]=peakdet(v, delta, x)
    %PEAKDET Detect peaks in a vector
    %        [MAXTAB, MINTAB] = PEAKDET(V, DELTA) finds the local
    %        maxima and minima ("peaks") in the vector V.
    %        MAXTAB and MINTAB consists of two columns. Column 1
    %        contains indices in V, and column 2 the found values.
    %      
    %        With [MAXTAB, MINTAB] = PEAKDET(V, DELTA, X) the indices
    %        in MAXTAB and MINTAB are replaced with the corresponding
    %        X-values.
    %
    %        A point is considered a maximum peak if it has the maximal
    %        value, and was preceded (to the left) by a value lower by
    %        DELTA.
    
    % Eli Billauer, 3.4.05 (Explicitly not copyrighted).
    % This function is released to the public domain; Any use is allowed.
    
    """

    max_tab = []
    min_tab = []
    
    cropped_df = df.loc[interval_offset:interval_offset + SAMPLE_RATE*4] # 3 was the original value
    cropped_df = cropped_df.reset_index(drop=True)

    data = cropped_df['Skin conductance']
       
    if x is None:
        x = np.arange(len(data))
    
    data = np.asarray(data)
    
    mn, mx = np.Inf, -np.Inf
    mn_pos, mx_pos = np.NaN, np.NaN
    
    look_for_max = True
    
    for i in np.arange(len(data)):

        this = data[i]
        if this > mx:
            mx = this
            mx_pos = interval_offset + x[i]
        if this < mn:
            mn = this
            mn_pos = interval_offset + x[i]

        if look_for_max:
            if this < mx-delta:
                max_tab.append((mx_pos, mx))
                look_for_max = False

        else:
            if this > mn+delta:
                min_tab.append((mn_pos, mn))
                mx = this
                mx_pos = interval_offset + x[i]
                look_for_max = True
                
    clean_max_tab = copy.deepcopy(max_tab)
    clean_min_tab = copy.deepcopy(min_tab)
    
    if len(min_tab) == 0:
        if len(max_tab) > 0:
            del clean_max_tab[0]
        provisional_value_x = interval_offset + 65
        provisional_value_y = cropped_df.loc[65][1]
        clean_min_tab.append((provisional_value_x, provisional_value_y))
        provisional_value_x = interval_offset + 85
        provisional_value_y = cropped_df.loc[85][1]
        clean_max_tab.append((provisional_value_x, provisional_value_y))
    
    if len(max_tab) == 0:
        if len(clean_min_tab) > 0:
            del clean_min_tab[0]
        provisional_value_x = interval_offset + 65
        provisional_value_y = cropped_df.loc[65][1]
        clean_min_tab.append((provisional_value_x, provisional_value_y))
        provisional_value_x = interval_offset + 85
        provisional_value_y = cropped_df.loc[85][1]
        clean_max_tab.append((provisional_value_x, provisional_value_y))

    # addressing when the peak is detected before the trough
    if clean_max_tab[0][1] >= clean_min_tab[0][1]:
        del clean_max_tab[0]

        if len(clean_max_tab) == 0:
            provisional_value_x = interval_offset + 85
            provisional_value_y = cropped_df.loc[85][1]
            clean_max_tab.append((provisional_value_x,provisional_value_y))
    
    if clean_min_tab[0][0] >= clean_max_tab[0][0]:
        del clean_max_tab[0]
        del clean_min_tab[0]
    
        provisional_value_x = interval_offset + 65
        provisional_value_y = cropped_df.loc[65][1]
        clean_min_tab.append((provisional_value_x, provisional_value_y))
        provisional_value_x = interval_offset + 85
        provisional_value_y = cropped_df.loc[85][1]
        clean_max_tab.append((provisional_value_x, provisional_value_y))
    
    if len(clean_min_tab) > len(clean_max_tab):
        del clean_min_tab[-1]
    
    if len(clean_max_tab) > len(clean_min_tab):
        del clean_max_tab[-1]
    
    if len(clean_min_tab) > 1:
        for k in range(len(clean_min_tab)):
            if not len(clean_min_tab) == 1:
                del clean_min_tab[-1]
                del clean_max_tab[-1]
        
    return np.array(clean_max_tab), np.array(clean_min_tab)


def get_min_max_dfs(sliced_df):
    global mean_one_sec
    global update_flag
    
    min_tab, max_tab = [], []
    mean_df = sliced_df.loc[:SAMPLE_RATE]
    mean_one_sec = mean_df['Skin conductance'].mean()
    offset_first_interval = SAMPLE_RATE*2
    offset_second_interval = SAMPLE_RATE*5
    offset_third_interval = SAMPLE_RATE*8
    
    max_tab_first_interval, min_tab_first_interval = peakdet(sliced_df, offset_first_interval, PEAK_DET_THRESHOLD)
    min_tab.extend(min_tab_first_interval)
    max_tab.extend(max_tab_first_interval)
    max_tab_second_interval, min_tab_second_interval = peakdet(sliced_df, offset_second_interval, PEAK_DET_THRESHOLD)
    min_tab.extend(min_tab_second_interval)
    max_tab.extend(max_tab_second_interval)
    max_tab_third_interval, min_tab_third_interval = peakdet(sliced_df, offset_third_interval, PEAK_DET_THRESHOLD)
    min_tab.extend(min_tab_third_interval)
    max_tab.extend(max_tab_third_interval)
    
    max_tab_df = pd.DataFrame(max_tab, columns=['sample', 'value'])
    min_tab_df = pd.DataFrame(min_tab, columns=['sample', 'value'])
    
    update_flag = False
    
    return min_tab_df, max_tab_df


def annotation_visualisation(sliced_df, idx, note=None):
    global ax
    global fig
    global max_tab_df
    global min_tab_df
    global min_markers_vars
    global max_markers_vars
    global points_x, points_y
    global dps
    
    """ Only plot"""

    if fig:
        ax.plot(sliced_df.index, sliced_df['Skin conductance'])
        plt.ylim(min(sliced_df['Skin conductance']) - OFFSET, max(sliced_df['Skin conductance']) + OFFSET)
        
        plt.axvline(x=SAMPLE_RATE, color='k', linestyle='-')
        ax.annotate('onset', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.115, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*2, color='r', linestyle='--')
        ax.annotate('onset + 1[s]', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.195, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*5, color='r', linestyle='--')
        ax.annotate('onset + 4[s]', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.425, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*7, color='k', linestyle='-')
        ax.annotate('offset', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.57,0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*8, color='r', linestyle='--')
        ax.annotate('offset + 1[s]', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.65, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*11, color='r', linestyle='--')
        ax.annotate('offset + 4[s]', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.8775, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )
        
        min_tab_df, max_tab_df = get_min_max_dfs(sliced_df)
        rects, markers_loc, min_markers_vars, max_markers_vars = creating_plots(min_tab_df, max_tab_df, ax)
        
        dps = []
        i = 0
        for point in rects:
            dp = DraggablePoint(point, i, markers_loc, min_markers_vars, max_markers_vars, ax)
            i += 1
            dp.connect()
            dps.append(dp)
        
    else:
        fig, ax = plt.subplots()
        ax.plot(sliced_df.index, sliced_df['Skin conductance'])
        plt.ylim(min(sliced_df['Skin conductance']) - OFFSET, max(sliced_df['Skin conductance']) + OFFSET)

        plt.axvline(x=SAMPLE_RATE, color='k', linestyle='-')
        ax.annotate('onset', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.115, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*2, color='r', linestyle='--')
        ax.annotate('onset + 1[s]', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.195, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*5, color='r', linestyle='--')
        ax.annotate('onset + 4[s]', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.425, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*7, color='k', linestyle='-')
        ax.annotate('offset', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.57, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*8, color='r', linestyle='--')
        ax.annotate('offset + 1[s]', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.65, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        plt.axvline(x=SAMPLE_RATE*11, color='r', linestyle='--')
        ax.annotate('offset + 4[s]', xy=(SAMPLE_RATE*2, ax.get_ylim()[1]),
                    xytext=(0.8775, 0.05), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )

        min_tab_df, max_tab_df = get_min_max_dfs(sliced_df)
        rects, markers_loc, min_markers_vars, max_markers_vars = creating_plots(min_tab_df, max_tab_df, ax)
        
        dps = []
        i = 0
        for point in rects:
            dp = DraggablePoint(point, i, markers_loc, min_markers_vars, max_markers_vars, ax)
            i += 1
            dp.connect()
            dps.append(dp)

            
def vis_event_data(sliced_df, idx):
    global fig1
    global ax1
    global ax2
    global ax3
    timestamp_series = sliced_df['Timestamp'] + pd.to_timedelta('1s')
    vline_marker_ts = timestamp_series[0]
    
    if fig1:        
        ax1.plot(sliced_df['Timestamp'], sliced_df['Skin conductance'], label='Skin Cond')
        ax2.plot(sliced_df['Timestamp'], sliced_df['Respiration'], label='Resp')
        ax3.plot(sliced_df['Timestamp'], sliced_df['Event'], label='Event')
        
        ax1.axes.get_xaxis().set_visible(False)
        ax2.axes.get_xaxis().set_visible(False)

        ax1.axvline(x=pd.to_datetime(vline_marker_ts), color='k', linestyle='-')
        ax1.annotate('onset', xy=(pd.to_datetime(vline_marker_ts), ax1.get_ylim()[1]),
                    xytext=(0.115, 0.15), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )
        ax2.axvline(x=pd.to_datetime(vline_marker_ts), color='k', linestyle='-')
        ax2.annotate('onset', xy=(pd.to_datetime(vline_marker_ts), ax2.get_ylim()[1]),
                    xytext=(0.115, 0.15), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )
        ax3.axvline(x=pd.to_datetime(vline_marker_ts), color='k', linestyle='-')
        ax3.annotate('onset', xy=(pd.to_datetime(vline_marker_ts), ax1.get_ylim()[1]),
                    xytext=(0.115, 0.15), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )
        
        ax1.legend(loc="upper right")
        ax2.legend(loc="upper right")
        ax3.legend(loc="upper right")
        
    else: 
        fig1, (ax1, ax2, ax3) = plt.subplots(3, figsize=(10, 3.25))
        ax1.plot(sliced_df['Timestamp'], sliced_df['Skin conductance'], label='Skin Cond')
        ax2.plot(sliced_df['Timestamp'], sliced_df['Respiration'], label='Resp')
        ax3.plot(sliced_df['Timestamp'], sliced_df['Event'], label='Event')
        
        ax1.axes.get_xaxis().set_visible(False)
        ax2.axes.get_xaxis().set_visible(False)

        ax1.axvline(x=pd.to_datetime(vline_marker_ts), color='k', linestyle='-')
        ax1.annotate('onset', xy=(pd.to_datetime(vline_marker_ts), ax1.get_ylim()[1]),
                    xytext=(0.115, 0.15), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )
        ax2.axvline(x=pd.to_datetime(vline_marker_ts), color='k', linestyle='-')
        ax2.annotate('onset', xy=(pd.to_datetime(vline_marker_ts), ax2.get_ylim()[1]),
                    xytext=(0.115, 0.15), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )
        ax3.axvline(x=pd.to_datetime(vline_marker_ts), color='k', linestyle='-')
        ax3.annotate('onset', xy=(pd.to_datetime(vline_marker_ts), ax1.get_ylim()[1]),
                    xytext=(0.115, 0.15), textcoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    )
        
        ax1.legend(loc="upper right")
        ax2.legend(loc="upper right")
        ax3.legend(loc="upper right")


class DraggablePoint:
    
    def __init__(self, pnt, i, markers_loc, min_markers_vars, max_markers_vars, ax):
        self.point = pnt
        self.press = None
        self.ax = ax

        self.lines = self.ax.lines[:1]
        self.tx = [self.ax.text(0, 0, "") for l in self.lines]
        self.marker = [self.ax.plot(markers_loc[i][0], markers_loc[i][1], marker="o", color="red")[0]]
        
        self.spans = []
        for peak_n in range(len(min_markers_vars)):
            self.vspan = ax.axvspan(min_markers_vars["min_marker{}".format(peak_n)],
                                    max_markers_vars["max_marker{}".format(peak_n)],
                                    facecolor='0.5')
            self.spans.append(self.vspan)
        
        
        self.draggable = False

    def connect(self):
        """Connect to all the events we need."""
        self.cidpress = self.point.figure.canvas.mpl_connect('button_press_event', self.click)
        self.cidrelease = self.point.figure.canvas.mpl_connect('button_release_event', self.release)
        self.cidmotion = self.point.figure.canvas.mpl_connect('motion_notify_event', self.drag)

    def click(self, event):       
        """Check whether mouse is over it; if so, store some data."""
        if event.inaxes != self.point.axes:
            return
        contains, attrd = self.point.contains(event)
        if not contains:
            return
        #print('event contains', self.point.xy)
        self.press = self.point.xy, (event.xdata, event.ydata)

        if event.button == 1:       # left click
            self.draggable = True
            self.update(event)

    def drag(self, event):
        """Move the rectangle if the mouse is over it."""
        if self.press is None or event.inaxes != self.point.axes:
            return
        (x0, y0), (xpress, ypress) = self.press
        dx = event.xdata - xpress

        if self.draggable:
            hght = round(self.update(event), 2)

        self.point.set_x(x0+dx)
        self.point.set_height(hght + OFFSET)
        self.point.figure.canvas.draw()        

    def release(self, event):
        """Clear button press information."""
        
        # Check if there was a click somewhere not close to the markers.
        # If it is close to the markers, the software will continue its process
        try:
            for self.span in self.spans:
                self.span.remove()

            self.press = None
            self.point.figure.canvas.draw()
            self.draggable = False

            for peak_n in range(len(min_markers_vars)):
                
                if min_markers_vars['min_marker{}'.format(peak_n)] == closest_x:
                    min_markers_vars['min_marker{}'.format(peak_n)] = x
                    
                    break
                elif max_markers_vars['max_marker{}'.format(peak_n)] == closest_x:
                    max_markers_vars['max_marker{}'.format(peak_n)] = x

            self.spans= []
            for peak_n in range(len(min_markers_vars)):
                self.vspan = ax.axvspan(min_markers_vars["min_marker{}".format(peak_n)],
                                             max_markers_vars["max_marker{}".format(peak_n)],
                                             facecolor='0.5')

                self.spans.append(self.vspan)
        
        except NameError:
            pass

        
    def update(self, event):        
        global x
        global closest_x

        for i, line in enumerate(self.lines):
            x, y = self.get_closest(line, event.xdata)

            try:
                for peak_n in range(len(min_markers_vars)):
                    if min_markers_vars['min_marker{}'.format(peak_n)] - 3 <= x <= min_markers_vars['min_marker{}'.format(peak_n)] + 3:
                        closest_x = min_markers_vars['min_marker{}'.format(peak_n)]
                        break
                    elif max_markers_vars['max_marker{}'.format(peak_n)] - 3 <= x <= max_markers_vars['max_marker{}'.format(peak_n)] + 3:
                        closest_x = max_markers_vars['max_marker{}'.format(peak_n)]
                        break

            except ValueError:
                closest_x = None
                continue

            self.tx[i].set_position((x, y))
            self.tx[i].set_text("x:{}\ny:{:.2f}".format(x, round(y,2)))
            self.marker[i].set_data([x], [y])
            hght = y        
        
        return hght

    def get_closest(self, line, mx):
        x, y = line.get_data()
        mini = np.argmin(np.abs(x - mx))
        return x[mini], y[mini]

    def disconnect(self):
        """Disconnect all callbacks."""
        self.point.figure.canvas.mpl_disconnect(self.cidpress)
        self.point.figure.canvas.mpl_disconnect(self.cidrelease)
        self.point.figure.canvas.mpl_disconnect(self.cidmotion)

        
def creating_plots(mintab_df, maxtab_df, ax):
    global markers_loc

    points_x = []
    points_y = []
    markers_loc = []
    for peak_idx in range(len(mintab_df['sample'])):
        points_x.append(int(mintab_df['sample'][peak_idx]))
        points_x.append(int(maxtab_df['sample'][peak_idx]))
        points_y.append(mintab_df['value'][peak_idx] + OFFSET)
        points_y.append(maxtab_df['value'][peak_idx] + OFFSET)
        min_marker = (int(mintab_df['sample'][peak_idx]), mintab_df['value'][peak_idx])
        max_marker = (int(maxtab_df['sample'][peak_idx]), maxtab_df['value'][peak_idx])
        markers_loc.append(min_marker)
        markers_loc.append(max_marker)

    rects = ax.bar(points_x, points_y, width=4, alpha=0.0)

    min_markers_vars = {}
    max_markers_vars = {}
    for peak_n in range(len(mintab_df)):
        min_markers_vars["min_marker{}".format(peak_n)] = mintab_df['sample'][peak_n]
        max_markers_vars["max_marker{}".format(peak_n)] = maxtab_df['sample'][peak_n]
        
    return rects, markers_loc, min_markers_vars, max_markers_vars


def change_event(button):
    global idx
    global sliced_df
    global fig
    global ax

    new_markers_loc = []

    # Export the last event data
    prev_event_data_df = save_df()
    export_df(prev_event_data_df)

    df_output.clear_output()
    with plot_output:

        if button.description=='Prev':
            if idx-1<0:
                ax1.cla()
                ax2.cla()
                ax3.cla()
                vis_event_data(sliced_dfs_list[0], idx)
                fig1.suptitle('Experiment data - Event #{}'.format(idx))
                for dp in dps:
                    dp.disconnect()
                plt.cla()
                annotation_visualisation(sliced_dfs_list[0], idx)
                fig.suptitle('Event #{} - This is the first event!'.format(idx))
        
            else:
                sliced_df = sliced_dfs_list[idx-1]
                idx -= 1
                ax1.cla()
                ax2.cla()
                ax3.cla()
                vis_event_data(sliced_df, idx)
                fig1.suptitle('Experiment data - Event #{}'.format(idx))
                for dp in dps:
                    dp.disconnect()
                plt.cla()
                annotation_visualisation(sliced_df, idx)
                fig.suptitle('Event #{}'.format(idx))
 
        elif button.description=='Next':
            if idx+1>=len(sliced_dfs_list):
                ax1.cla()
                ax2.cla()
                ax3.cla()
                vis_event_data(sliced_dfs_list[-1], idx)
                fig1.suptitle('Experiment data - Event #{}'.format(idx))
                for dp in dps:
                    dp.disconnect()
                plt.cla()
                annotation_visualisation(sliced_dfs_list[-1], idx)
                fig.suptitle('Event #{} - This is the last event!'.format(idx))
                
            else:
                sliced_df = sliced_dfs_list[idx+1]
                idx += 1
                ax1.cla()
                ax2.cla()
                ax3.cla()
                vis_event_data(sliced_df, idx)
                fig1.suptitle('Experiment data - Event #{}'.format(idx))
                for dp in dps:
                    dp.disconnect()
                plt.cla()
                annotation_visualisation(sliced_df, idx)
                fig.suptitle('Event #{}'.format(idx))

                
def start_process(button):
    global sliced_df
    with plot_output:
        if button.description=='Start Annotation':
            plt.rcParams['figure.figsize'] = (19, 4.5)
            sliced_df = sliced_dfs_list[0]
            vis_event_data(sliced_df, idx)
            fig1.suptitle('Experiment data - Event #{}'.format(idx))
            fig1.canvas.header_visible = False
            plt.show()
            annotation_visualisation(sliced_df, idx)
            fig.suptitle('Event #{}'.format(idx))
            fig.canvas.header_visible = False
            plt.show()
            
            button.layout.visibility = 'hidden'
            box = widgets.HBox(children=[start_button, prev_button, next_button, reset_button, visualise_annotation_button])
            display(box)

            
def reset_plot(button):
    new_markers_loc = []

    df_output.clear_output()
    with plot_output:
        if button.description=='Reset':
            sliced_df = sliced_dfs_list[idx]

            for dp in dps:
                dp.disconnect()
            plt.cla()
            annotation_visualisation(sliced_df, idx)


def visualise_df(button):
    global df, df1
    global full_df
    global updated_min_tab_df, updated_max_tab_df
    global signal_values
    global cols_list
    
    troughs_peaks_col_list = ['Trough_1_x', 'Trough_1_val', 'Peak_1_x', 'Peak_1_val', 'Trough_2_x', 'Trough_2_val', 'Peak_2_x', 'Peak_2_val', 'Trough_3_x', 'Trough_3_val', 'Peak_3_x', 'Peak_3_val']
    diff_cols_list = ['Diff_1', 'Diff_2', 'Diff_3']
    
    updated_min_tab_values = []
    updated_max_tab_values = []
    updated_x_min_tab_markers_values = []
    updated_x_max_tab_markers_values = []
    
    for item in min_markers_vars.values():
        updated_x_min_tab_markers_values.append(item)

    for item in max_markers_vars.values():
        updated_x_max_tab_markers_values.append(item)

    for x_value in updated_x_min_tab_markers_values:
        y_value = sliced_df.loc[x_value][1]
        updated_min_tab_values.append((int(x_value), y_value))

    for x_value in updated_x_max_tab_markers_values:
        y_value = sliced_df.loc[x_value][1]
        updated_max_tab_values.append((int(x_value), y_value))
    
    updated_min_tab_df = pd.DataFrame(updated_min_tab_values, columns=['sample', 'value'])
    updated_max_tab_df = pd.DataFrame(updated_max_tab_values, columns=['sample', 'value'])
    
    df_output.clear_output()
    
    with df_output:
        if button.description=='Visualise Annotation':
            event_df = get_df(updated_min_tab_df, updated_max_tab_df)
            display(event_df)
            

def get_df(updated_min_tab_df, updated_max_tab_df):
    troughs_peaks_col_list = ['Trough_1_x', 'Trough_1_val', 'Peak_1_x', 'Peak_1_val', 'Trough_2_x', 'Trough_2_val', 'Peak_2_x', 'Peak_2_val', 'Trough_3_x', 'Trough_3_val', 'Peak_3_x', 'Peak_3_val']
    diff_cols_list = ['Diff_1', 'Diff_2', 'Diff_3']
    
    signal_values = []
    for k in range(int(len(troughs_peaks_col_list)/4)):
        # trough values
        signal_values.append([updated_min_tab_df.loc[k][0]])
        signal_values.append([updated_min_tab_df.loc[k][1]])
        # peak values
        signal_values.append([updated_max_tab_df.loc[k][0]])
        signal_values.append([updated_max_tab_df.loc[k][1]])

    df = pd.DataFrame(np.transpose(signal_values), columns=troughs_peaks_col_list)
    df['event'] = [idx]
    df = df.set_index('event')

    df.insert(loc=0, column='Mean', value=mean_one_sec)
    df1 = df.copy(deep=True)

    # Calculate the individual troughs and peaks difference
    cnt = 2
    difference_list = []

    for j in range(3):
        trough_cnt = cnt
        peak_cnt = cnt + 2 

        if df.loc[idx][peak_cnt] >= 25:
            df.loc[idx][trough_cnt] = -99
            df.loc[idx][peak_cnt] = -99
            difference = -99
        elif df.loc[idx][trough_cnt] == 0:
            df.loc[idx][trough_cnt] = -99
            df.loc[idx][peak_cnt] = -99
            difference = -99
        elif df.loc[idx][trough_cnt] >= df.loc[idx][peak_cnt]:                           
            df.loc[idx][trough_cnt] = 0
            df.loc[idx][peak_cnt] = 0
            difference = 0
        else:
            difference = df.loc[idx][peak_cnt] - df.loc[idx][trough_cnt]
            if difference <= DIFF_TRSHD:
                df.loc[idx][trough_cnt] = 0
                df.loc[idx][peak_cnt] = 0
                difference = 0

        difference_list.append([difference])
        cnt += 4

    # Create the diff data frame to concatenate it side to side with the troughs and peaks' data frame
    diff_df = pd.DataFrame(np.transpose(difference_list), columns=diff_cols_list)
    diff_df = diff_df.set_axis([idx])
    # concatenate
    full_df = pd.concat([df, diff_df], axis=1)
    
    return full_df


def save_df():

    updated_min_tab_values = []
    updated_max_tab_values = []
    updated_x_min_tab_markers_values = []
    updated_x_max_tab_markers_values = []
    
    for item in min_markers_vars.values():
        updated_x_min_tab_markers_values.append(item)

    for item in max_markers_vars.values():
        updated_x_max_tab_markers_values.append(item)

    for x_value in updated_x_min_tab_markers_values:
        y_value = sliced_df.loc[x_value][1]
        updated_min_tab_values.append((int(x_value), y_value))

    for x_value in updated_x_max_tab_markers_values:
        y_value = sliced_df.loc[x_value][1]
        updated_max_tab_values.append((int(x_value), y_value))
    
    updated_mintab_df = pd.DataFrame(updated_min_tab_values, columns=['sample', 'value'])
    updated_maxtab_df = pd.DataFrame(updated_max_tab_values, columns=['sample', 'value'])
    
    event_df = get_df(updated_mintab_df, updated_maxtab_df)
    
    if saved_df_list:
        if event_df.index[0] in idx_list:
            idx_to_del = idx_list.index(event_df.index[0])
            del saved_df_list[idx_to_del]
            del idx_list[idx_to_del]
            idx_list.append(event_df.index[0])
            saved_df_list.append(event_df)

        else:
            idx_list.append(event_df.index[0])
            saved_df_list.append(event_df)

    else:
        idx_list.append(event_df.index[0])
        saved_df_list.append(event_df)
        
    return saved_df_list
        

def export_df(saved_df_list):
    exported_df = pd.concat(saved_df_list)
    exported_df.sort_index(inplace=True)
    output_filename_path = os.path.join(OUTPUT_FOLDER, 'annotated_' + filename + '.csv')
    exported_df.to_csv(output_filename_path, index_label='event_number')


def text_input_callback(participant_id):
    global plot_output
    global df_output
    global sliced_dfs_list
    global idx
    global start_button
    global prev_button
    global next_button
    global reset_button
    global visualise_annotation_button
    global filename
    
    if participant_id == "":
        pass

    else:
        try:
            filename = participant_id
            filename_path = os.path.join(DATASET_FOLDER, filename + '.mat')

            biopac_mat = loadmat(filename_path)
            print('Loading and processing the data from {}. Please wait a few seconds...'.format(filename))

            # extract sensors data
            biopac_data = biopac_mat['data']
            # create the pandas dataframe
            biopac_df = pd.DataFrame(biopac_data, columns = biopac_mat['labels'])
            targeted_biopac_df = biopac_df.drop(columns=['Blink', 'Corrugator', 'Zygomaticus', 'ECG', 'Self report',
                                                        'Startle', 'Open', 'Tone', 'Pulse', 'Shock'])

            time_now = pd.to_datetime("2000-1-1")

            timestamps_df = []
            timestamps_df.append(time_now)
            for i in range(1, biopac_df.shape[0]):
                time_now = time_now + pd.to_timedelta('1ms')
                timestamps_df.append(time_now)

            # insert the timestamps to the dataframe
            targeted_biopac_df.insert(0, "Timestamp", timestamps_df, True)
            ts_targeted_biopac_df = targeted_biopac_df.set_index('Timestamp')

            # Down sampling data
            downsampled_df = pd.DataFrame()
            downsampled_df['Skin conductance'] = ts_targeted_biopac_df['Skin conductance'].resample(SAMPLE_RATE_STRING).mean()
            downsampled_df['Respiration'] = ts_targeted_biopac_df['Respiration'].resample(SAMPLE_RATE_STRING).mean()
            downsampled_df['Event'] = ts_targeted_biopac_df['Event'].resample(SAMPLE_RATE_STRING).mean()

            # Filtering data
            events = downsampled_df['Event']
            threshold = 0.0
            filtered_events = downsampled_df['Event'] > threshold
            downsampled_df.insert(len(downsampled_df.columns), "Event Filter",  filtered_events, True)

            # Create "change" column. This detects the change of values, i.e. when an event started
            downsampled_df["Event Change"] = downsampled_df["Event Filter"].shift() != downsampled_df["Event Filter"]

            # Set the first value as False. As there is no event change. 
            downsampled_df["Event Change"].iloc[0] = False

            event_change_idx =  downsampled_df.index[downsampled_df['Event Change']].tolist()

            chunks_list = []
            num_events = int((len(event_change_idx)) / 2)

            for k in range(num_events):
                if k == 0:
                    first_value = k
                    second_value = first_value + 1
                    chunks_list.append([event_change_idx[first_value], event_change_idx[second_value]])
                else:
                    first_value = second_value + 1
                    second_value = first_value + 1
                    chunks_list.append([event_change_idx[first_value], event_change_idx[second_value]])

            chunks_df = pd.DataFrame(chunks_list, columns=['Event_start', 'Event_end'])

            # Create different subsets related to events
            sliced_dfs_list = []
            if os.path.exists('./csv_data/' + filename):
                pass

            else:
                os.mkdir('./csv_data/' + filename)

            for event_number in range(len(chunks_df)):
                event_timestamps = chunks_df.iloc[event_number]
                annotation_start = event_timestamps[0] - pd.to_timedelta('1s')
                annotation_end = event_timestamps[1] + pd.to_timedelta('5s')
                sliced_df = downsampled_df[annotation_start:annotation_end]
                sliced_df = sliced_df.reset_index()
                sliced_df = sliced_df.drop(columns=['Event Filter','Event Change'])

                sliced_df.to_csv('./csv_data/{}/event_number_{}.csv'.format(filename,event_number+1), index=False)    
                sliced_dfs_list.append(sliced_df)

            sliced_df = sliced_dfs_list[0]

            # Widgets 
            start_button = widgets.Button(description='Start Annotation')
            prev_button = widgets.Button(description='Prev')
            next_button = widgets.Button(description='Next')
            reset_button = widgets.Button(description='Reset')
            annotate_button = widgets.Button(description='Annotate')
            save_annotation_button = widgets.Button(description='Save Annotation')
            visualise_annotation_button = widgets.Button(description='Visualise Annotation')
            export_annotation_button = widgets.Button(description='Export Annotation')

            idx = 0
            plot_output = widgets.Output()
            df_output = widgets.Output()

            display(start_button)
            display(plot_output)
            display(df_output)

            start_button.on_click(start_process)
            prev_button.on_click(change_event)
            next_button.on_click(change_event)
            reset_button.on_click(reset_plot)
            visualise_annotation_button.on_click(visualise_df)

        except FileNotFoundError as not_found:
            print("Sorry, the filename '{}' does not exist. Please verify the filename and try again.".format(not_found.filename))
            pass

# Main  
fig = None
fig1 = None
plot_output = None
closest_x = 0

xs_values = []
ys_values = []
saved_df_list = []
idx_list = []
new_markers_loc = []

participant_ids = ['Ext_P303', 'Acq_P301']

# Loading data
text_box = widgets.Text(
    value='',
    placeholder='Paste the participant filename here',
    description='String:',
    disabled=False,
    layout = widgets.Layout(width='350px')
)

interact(text_input_callback, participant_id=text_box);

interactive(children=(Text(value='', description='String:', layout=Layout(width='350px'), placeholder='Paste t…